In [1]:
%run forecasting.ipynb

100%|██████████| 5/5 [00:01<00:00,  2.70it/s]
Importing plotly failed. Interactive plots will not work.
  0%|          | 0/5 [00:00<?, ?it/s]

Initial log joint probability = -23.8441
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      94       2304.99    0.00059656       212.127    4.31e-06       0.001      145  LS failed, Hessian reset 
      99       2305.17   0.000825136       89.4793      0.9746      0.9746      150   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     193        2305.9    0.00037808       161.031   2.206e-06       0.001      316  LS failed, Hessian reset 
     199       2306.19    0.00069264       73.7016      0.3052           1      323   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     243       2306.29   4.25779e-05       86.2633   5.088e-07       0.001      415  LS failed, Hessian reset 
     271       2306.29   1.51602e-07       79.5501           1           1      452   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is

 20%|██        | 1/5 [00:02<00:11,  2.84s/it]

Initial log joint probability = -34.5062
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2717.63   0.000201329       78.4211      0.7713      0.7713      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2721.23   0.000657679       64.9873      0.4638           1      262   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     266       2721.27   2.03817e-05       61.8377   4.043e-07       0.001      393  LS failed, Hessian reset 
     299       2721.27   6.35448e-06       61.8897       1.593      0.4364      432   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     365       2721.28   1.48875e-07       76.8861      0.3067           1      514   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


 40%|████      | 2/5 [00:04<00:06,  2.28s/it]

...

 60%|██████    | 3/5 [00:06<00:04,  2.02s/it]

Initial log joint probability = -31.3194
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2183.42    0.00301576       166.397           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     150       2185.59   0.000842076       121.559   1.067e-05       0.001      240  LS failed, Hessian reset 
     199       2186.15   2.85771e-05       72.1154           1           1      305   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     265       2186.18   0.000178036       84.1751   2.163e-06       0.001      427  LS failed, Hessian reset 
     299       2186.21   2.64698e-06       62.9735      0.2269           1      472   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     328       2186.21    1.2895e-07       57.2724      0.5866      0.5866      510   
Optimization terminat

 80%|████████  | 4/5 [00:08<00:01,  1.96s/it]

Initial log joint probability = -20.2419
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      90       2185.31    0.00243635       240.077    2.03e-05       0.001      151  LS failed, Hessian reset 
      99       2185.43   0.000117324        88.201      0.5176      0.5176      162   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     129       2185.45   1.56974e-07       61.6273      0.4624      0.4624      206   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


  0%|          | 0/5 [00:00<?, ?it/s]INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:NP.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO:NP.config:Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 170
INFO:NP.config:Auto-set epochs to 170
 94%|█████████▎| 239/255 [00:01<00:00, 150.39it/s]
INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.11E-02, min: 1.13E+00
INFO:NP.utils_torch:lr-range-test results: steep: 3.11E-02, min: 1.13E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.40E-01
INFO:NP.utils_torch:learning rate range test selected lr: 3.40E-01
100%|██████████| 5/5 [00:53<00:00, 10.78s/it]


In [2]:
kaggle_test = pd.read_csv("../data/test.csv")
kaggle_test['store_item'] = kaggle_test.store.astype(str) + '-' + kaggle_test.item.astype(str)
kaggle_test = kaggle_test.set_index(pd.to_datetime(kaggle_test.date))
kaggle_test.drop(['id', 'store', 'item', 'date'], axis=1, inplace=True)

In [3]:
testing_models = False
forecast_horizon = 90

if testing_models:
    kaggle_test = kaggle_test[kaggle_test.store_item.isin(['1-1', '1-2', '1-3'])]
    data = data[data.store_item.isin(['1-1', '1-2', '1-3'])]

In [4]:
horizon_end_date = kaggle_test.index[-1]
fcast_begin = kaggle_test.index[0]

In [5]:
kaggle_test["autoreg"] = fit_autoreg(df=data, fcast=fcast_begin, horizion_end=horizon_end_date)

100%|██████████| 500/500 [01:13<00:00,  6.83it/s]


In [6]:
kaggle_test["exp_smooth"] = exp_smooth_predictor(df=data, seas="add", fcast=fcast_begin, horizion_end=horizon_end_date)

100%|██████████| 500/500 [12:24<00:00,  1.49s/it]


In [7]:
kaggle_test["ardl"] = ardl_predictor(data, fcast=fcast_begin, horizion_end=horizon_end_date)

In [8]:
date_list = [kaggle_test.index[-1] + timedelta(days=x+1) for x in range(forecast_horizon)]

kaggle_test["xgb_preds"] = xgb_predictor(data, date_list)

100%|██████████| 500/500 [04:05<00:00,  2.04it/s]


In [ ]:
kaggle_test["prophet"] = prophet_predictor(data, date_list)

In [ ]:
kaggle_test["neural_prophet"] = neural_prophet(data, date_list)

In [11]:
if not testing_models:
    from datetime import date
    today = date.today()
    kaggle_test.to_csv(f"../data/predictions/predictions-final-{today.month}-{today.day}.csv")